In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True,stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [4]:
import os
from Bio import SeqIO
import subprocess

family_group = "8.Lycaenidae"

output_location = f"/mnt/g/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Maniola_hyperantus"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
            print(left_oh, right_oh)
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            query_name = coordinate_line_split[6]
            exon = f"Exon_{query_name.split('_')[-1]}"
            error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            continue
            
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "16":
            if complement == 1:
                translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                print(gene_sequence)
            else:
                translated = gene_sequence[int(left_oh):].translate()
                print(gene_sequence)

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Aricia_agestis
GCF_905147365.1_ilAriAges1.1_genomic.fna
21824941 21825037
0 1
Getting Gene


/home/saurav/miniconda3/envs/genomic_work/lib/python3.11/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


MAWVLRTPQIHSSFSNLGFSHPDGYHINDNCN
>Aricia_agestis_Exon_1_NC_056406.1_21824941_21825037_left_0_right_1
ATGGCGTGGGTCCTTCGCACCCCCCAAATACATAGCTCATTCAGTAATCTCGGATTTTCACACCCTGATGGATATCACATAAACGATAATTGCAATG
21826059 21826609
2 0
Getting Gene
>Aricia_agestis_Exon_2_NC_056406.1_21826059_21826609_left_2_right_0
CGGCCCTGGAGTTAATTCTGCATAATATACTAACAGAAGACAAATTACAACGAACCTACCGCAGAAGTATTCGATTCGGACAAAACATAAATAAAGATCTCATTCCCCTGCTGATTCATACCAAAGACGAGAAGACCGTCGAACTTTTGATAAAGATATTCGTTAATTTGTCGATACCCGTCGAGTGCCTCCTCTCGCTAGAGACGACCACCAAAACAGATTTCGGTAGACACGCAATTTTCGAAATAAATAGCTTGCTCGCTTCGACTAAAACGACTCTAACCGACCATCGAGCCACGAAAGTGGTGATcgaatttttaaagaaaaactcCGAATCCGGACCCAACGGCAAGCTGTCGATGGAACAGCGCACCAACATCTGCAACACGCTCCTGTTCTTGCGGAACGTGTTGCACATACCGGAGGAAGTGAACGTCCTGTCTTGTAGCTATAGCGGACCATCTCACTCGGTTCAAAATCAAATACTGTGGAATTTGTTCAGTCATAGTTTTGACAAAATTCTACTCAAACTTATTACGATCCCGGACGCG
21827200 21827353
0 1
Getting Gene
>Aricia_agestis_Exon_3_NC_056406.1_21827200_21827353_left_0_right_1
GTGCACTGGGCAGTGACCATGGTACAATTGATAGCATTGTTGT

23851421 23851704
2 0
Getting Gene
>Helleia_helle_Exon_4_OY971414.1_23851421_23851704_left_2_right_0
GTAGCAATCAATCGTCACCTATTTTAACATCTAACCCAACATCAGACTCATCTGATAATGCCGGCAGGGGAAAAAGTGACGAAGAACGTAATTCGACCGATACATGGGAAAGTGTATCGGGCGcaggtaaagaaaataattatgaaaagcATAAACCATCCGAAACTGGTGAACCATCCGAAAAAAATCTCAAGAAACCGTCtgaaaatgaaacaaatttcaAGAAAGATGGCAATATGAAAATATCGTCTGGAAATAGCGCAAGTACCGGATATACAAATTAT
23839851 23840077
0 2
Getting Gene
>Helleia_helle_Exon_5_OY971414.1_23839851_23840077_left_0_right_2
GATTTGTTGGCCAGTGAATCGgctaatacaaaaaatacatatccTGAGAAAAGGAATAAATGGAAGAAATATTTTACCAATCGAATTGTGAACCAAGAAACCACGACCACGTCTAGTAACGAAGACGAAttgtcaaataaaaatcaaCACAAGTCTAAATCCCGCGCAAATCTTACTGCTCCAGAACGTAGGAGGAAAAAAATTGTGAAGCGTGGAAAATTAAACAT
23839228 23839588
1 0
Getting Gene
>Helleia_helle_Exon_6_OY971414.1_23839228_23839588_left_1_right_0
AATAAACGCAAATGGAGTTACTCATATGTCGCCAACAGACGATAACATATCAAATTTGCTAAAAGAATTCACGATTGAGTTTTTATTCAAAGGCTTCAACTCGCTGGTACAGACGCTTCATAGGCAAATAATATCTAACGTGGGAGTAGAGATTGACACATCGCACTTCTTCTGGCTAGTTACA

18456074 18456274
2 1
Getting Gene
>Lycaena_phlaeas_Exon_12_HG995163.1_18456074_18456274_left_2_right_1
tctTTAATCAATCGGTGCCGCTTGTGCCGTGGAATTGCCAACAGGCCGCTGTTTGTAAGGACCTTAAATTCCTACATCTGCTCAATAAGCTAGGCTTTCAAATGCCAGTCGACACCGGCAAAGTCTTCATTCGGATACCGAACTTTTGGACGGCTGATTATTTGTATGAAGTGGCTGGAAAAATTTCACCCATAGACCCCA
18455648 18455816
2 2
Getting Gene
>Lycaena_phlaeas_Exon_13_HG995163.1_18455648_18455816_left_2_right_2
CTAAACTGAGGTTCTCCTTGAGCGATATATCTCAGAGCAGCGTCCAGTCGTCCAGCGAGCGGCCGTGGCTGGATTCGCGCCAGTTCGGTCCGCCGCCGTCCACCGCAGAGATCTTCTTCCAAGTGCACAAGGAACAACACCTCGCCAACATACTCAACTTTACGCCAAT
18454392 18454490
1 2
Getting Gene
>Lycaena_phlaeas_Exon_14_HG995163.1_18454392_18454490_left_1_right_2
GCCGGGATCGTCGTTTGATTCGACTGTTGTGAACAAACCTAACTGGTTGGAGCTGGTACAAAAATCTCAAGAGTACAAAATTACCTTGGACTTGGAAAG
18450234 18450374
1 2
Getting Gene
>Lycaena_phlaeas_Exon_15_HG995163.1_18450234_18450374_left_1_right_2
GTCGATAGTGGAGACCAACAAGCATGTGGAGGTGGTGGAGCAGTTCCCGCGGTCGGCGAGCCCGCCCCCCTCCTCCCCCTTCGCGCTCTCCCCCGCGGACCCCAAGGACCCG

...

31642719 31642919
2 1
Getting Gene
>Phengaris_arion_Exon_12_OY751432.1_31642719_31642919_left_2_right_1
TTTTCAATCAATCCGTGCCTCTTGTCCCGTGGAACTGTCAGCAGGCGGCCATATGTGGAGACCTCAAGTTTCTCCAGCTGATCCACAAGCTGGGCTTCTATATGCCGGTCGACACTGGCAAAATGTTTATACGGATACCTAATTTTTGGACGGCAGACGTTTTGTATGAGGTTGCGGCAAAAATTACACCAATAGACTCCT
31642118 31642292
2 2
Getting Gene
>Phengaris_arion_Exon_13_OY751432.1_31642118_31642292_left_2_right_2
cTAAGCTGAAATTCTCTCTGAGTGACATATCTCAGAGTAGTATTCAGAATTCGAAGAGAGACTATACGCCGTCGGTTGGGGGGAGCAAAAACGCGGTCATACCTTCTACACCTGAGAATCATTATCAAGTGCACAAGGAAAAACATCTCGCCAACATCCTCAACTTCACACCTAT
31641892 31641990
1 2
Getting Gene
>Phengaris_arion_Exon_14_OY751432.1_31641892_31641990_left_1_right_2
GCCCGGATCATCTTTCAATCCGGTTCATGTGAACAAACCAAATTGGCTGGAGTTGGTCAGAAAATCTCAAGAGTATACAATTACGCTGGACTTAGAGAA
31639919 31640089
1 2
Getting Gene
>Phengaris_arion_Exon_15_OY751432.1_31639919_31640089_left_1_right_2
GTCGATCGTGGAAACGAACAAGAACCCGGCGCAAGCGAAGGCGGGCAAGTGCgtgcgggcggcgcgggcgacACGTGCGGCGTCCGGGGCGGACGACGCGCCGCCG

6979572 6979795
2 0
Getting Gene
>Polyommatus_icarus_Exon_4_OW569321.1_6979572_6979795_left_2_right_0
GCAGTGATCATTCCTCGCCTATCTTAACGTCGGACCCGAATTCAGACTCGTCCGATACTGTCGGTAGCGGAAAAAGTGACGAGGATATGCCCAATAACACTTGGGAGAACACGACCACAGAAAACAAAGAGACCAACGATGAAACCCAGCAGAAGGACATGGATGGAAAGTCAACGTACACCAGCAGCGAGAACGACCACAATATTAAAGGGTCATCTGAAAGC
6979028 6979272
0 2
Getting Gene
>Polyommatus_icarus_Exon_5_OW569321.1_6979028_6979272_left_0_right_2
GATTTGCTCGCGAAAGACACCGTCACGAAGGAAAAATTATATTCCGACAAAAAGAACAAGCGGAATATAGAATTTATCAGTCGGCGAAAGAATCAGGAGACGACGACGTCCAGCAACGAGGACGAGCTGCCGGCCAAGAAGCCGCACAACTCCAAACCCCGCATGGCCGGCAAACAGCGCCCGCCCAGCGCCACGCCCCAggacagaaagagaaagaagATCGTCAAACGTGCAAAGTTAAACAT
6978436 6978796
1 0
Getting Gene
>Polyommatus_icarus_Exon_6_OW569321.1_6978436_6978796_left_1_right_0
ACTAAATGCAAAAGGACTAACGCAAACCTCGCCGACGGACGACGACATATCCATCGTATTGAAGGAGTTCACGGTCTGTTTCCTGCTGAAGGGCTACAACTCCCTAGTGCGGACTCTTCACTCGCAAATCCTCACTAATATAGAACTGGAAATCGACACTTCGCATTTCTTCTGGCTGGTGACATATTTCTTGAAGTTCGCGACTCAGATCGAGTTGGACTTG

>Polyommatus_iphigenia_Exon_12_OY730189.1_26878945_26879145_left_2_right_1
TTTTCAATCAGTCCGTGCCGCTGGTGCCGTGGAACTGTCAGCAGGCGGCGATCTGCGGCGACCTCAAGTTCCTGCAGCTGATACACAAGCTGGGCTTCGCCATGCCGGTCGACACCGGGAAACTGTTCATACGGATACCGAACTTCTGGACGGCCGACGTGCTGTATGAAGTCGCTGGAAAAATCACGCCCATAGACCCCT
26879822 26879993
2 2
Getting Gene
>Polyommatus_iphigenia_Exon_13_OY730189.1_26879822_26879993_left_2_right_2
CGACGCTGAAGTTCTCCCTGAGCGACATATCACAGAGCAGTATTCAGAAGTCGAGGCGCGACTACGCGCCGGCGGTGGGAGCGAGCAAGGCGGACATGCTGCCCGCTGCAGACAACCACTACCAAGTACACAAGGAGAAACATCTCGCCAACATACTCAACTTCACTCCCAC
26880495 26880593
1 2
Getting Gene
>Polyommatus_iphigenia_Exon_14_OY730189.1_26880495_26880593_left_1_right_2
GCCTGGATCATCTTTCAATCCTGTTCAAGAAACTAAACCTAACTGGCTGGATTTGGTCAAAAAAGCTCAAGAGTATACAATCACGCTGGATTTAGAAAA
26880844 26881005
1 2
Getting Gene
>Polyommatus_iphigenia_Exon_15_OY730189.1_26880844_26881005_left_1_right_2
GTCGATCGTGGAGACAAACATAAATCTGGAGCAGGCGAAGTCGAGCGAATGCGTGCGGTCGCGCAGCTCGCGGGCGGGGAGCTCGCTGGGCGAGGCCGACGAGGACCCGCCTCCTGACGA

>Calephelis_virginiensis_Exon_4_NJDC01002706.1_51594_51835_left_2_right_0
GGAGTGATGATTCTTCACCCAATTTAACGTCGGATCCAACATCAGATTCATCAGACACTGGAGCCAGTGGTAAAATTAACACACAGAACCCGGCGAACGGTGTATGGAATAACAATGAAAGTAAGGAGGGACATAAcaatgagaaaattgaaaaagaatcTCAACCCAATTCTAATGAGTCGGGGTCCAACTCTACTGGATCTGAGTCTAAATCGAAGGAAGAGAGTAACAGTGAAAGCAAG
50598 50896
0 2
Getting Gene
>Calephelis_virginiensis_Exon_5_NJDC01002706.1_50598_50896_left_0_right_2
GgtagtattttaaatactaatatacaaaatgaaGCTAGCAGCATTACTTCCAAAAACAACTCAACTACTACAAGTGAAAAGAGAGGGTCTTCCGAAAATTCTGATTGCGGTTATGGTACTCAGATTGAAAATCAAGAAATCATATCAACCTCAAGCAACGAAGACGAAGTGCCAGCAAAGAAACCAATGCATCAGAAACCGCACAATCCTAAACAAAGGACAAATAATAAAGTGCGCTCAAATGTTTCGCTTCAAGAGCGCAAgaggaaaaaaattgtaagaaggggaaaaacaaatat
48762 49122
1 0
Getting Gene
>Calephelis_virginiensis_Exon_6_NJDC01002706.1_48762_49122_left_1_right_0
AATAAACGTCCAAGGACTGCTGCACAGCACACCAACAGATGACGACATATCCAATGTTCTAAAAGAATTCACTGTGGATTTCCTTTTGAAAGGATACAACTCGTTAGTACAGACGCTTCATTCACAAATACTAACAAACCTTCAACTAGAAATCGATACGTCGCATTTCTTTTGGCT

>Eumaeus_atala_Exon_12_JAFELO010000109.1_232541_232741_left_2_right_1
TTTATAATCAACCAGTGCCGCTTGTGCCGTGGAACTGCCAGCAGGCGGCCATATGCGGCGACCTCAAATTTCTGCAATTCATTCATAAGCTGGGATTTTATATTCCTGTCGACACTGGGAAAATGTTTATTCGGATACCCAACTTTTGGACGGCCGACTTTTTatacgaagtcgcgggtaaaaTTTCGACTATAGACACCT
232280 232457
2 2
Getting Gene
>Eumaeus_atala_Exon_13_JAFELO010000109.1_232280_232457_left_2_right_2
caAAACTGAAATTCTCCCTGAGCGATATATCTCAGAGCAGCGTCCAGATGTCCAGAGAGCCTTATAACCCATCGCTTGTAGCACGCAACATTCCGATCTCGCCGTCGACGACACCCGAGAACTACTACCAAGTGCACAAGGAGAAACACTTTGCTAACATACTGAACTTCACGccaat
231622 231723
1 2
Getting Gene
>Eumaeus_atala_Exon_14_JAFELO010000109.1_231622_231723_left_1_right_2
gCCTGGAACGTCGTTTAATTCAATTCATGCAAATAGACAGCATAATTGGTTAGACTTGGTTAAGAAGTCTCAAGAATATACAATTACTTTGGACTTAGAAAA
229686 229835
1 2
Getting Gene
>Eumaeus_atala_Exon_15_JAFELO010000109.1_229686_229835_left_1_right_2
GTCAGTAGTGGAGACAAAGGAGAACCCAGAGAAGTTCAAGTGTGAGAAGCTGTCCCGCGCCTCGCGCAGTCCGTCACCCGCCTGCCTGTCGCCGCGCTCGCAGGACAAGGGCGACGCCCAACAGACCGACTTCGAGTGCAACA